# Meta
In this notebook we will save out the brain, skull, and fMRI activations to then align in blender

# Setup

In [1]:
from notebook_viewer_functions import *
from functions import *
import numpy as np
import ants
import matplotlib.pyplot as plt
import os

proj_root = parent_directory()
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"
cache_folder = "/Volumes/GlyphA_R1/nvol_cache"
template_folder =  os.path.join(proj_root, "templates/")
output_folder = os.path.join(proj_root, "output/blender_alignment")

bold_image = ants.image_read(bold_stim_filepath)
t1_image = ants.image_read(t1_input_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_template = ants.image_read(mni_anat_filepath)
mni_mask = ants.image_read(mni_mask_filepath)

In [2]:
normalize = (lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr)))

In [ ]:
mask = generate_brain_mask(t1_image, mni_template, mni_mask)

In [ ]:
mask_vol = create_normalized_volume(mask.numpy())

In [32]:
og_t1_vol = create_normalized_volume(t1_image.numpy())

In [31]:
og_bold_vol = create_normalized_volume(bold_image.numpy())

# Skull Stripping and Thresholding

## Thresholding BOLD

In [17]:
#threshold = (lambda arr, thresh=0.1: np.where(arr <thresh, 0.0, arr))
def threshold(arr, cutoff=0.3):
    return np.where(arr<cutoff, 0.0, arr)

Our bold skull stripping function isn't working too great, so let's just threshold it

Of note, the color ramping in blender doesn't work. **Once it's there, the eyes are the most dense part, which seems to imply some data shifting when we change to vdb!**

In [24]:
explore_3D_vol(og_bold_vol[30,:,:,:], dim='z')

interactive(children=(IntSlider(value=92, description='slice', max=184), Output()), _dom_classes=('widget-inte…

It seems to get rid of all the cerebellum and brain stem! But we'll still use this for testing

In [44]:
bold_threshed_vol = threshold(og_bold_vol, cutoff=0.1)
explore_3D_vol(bold_threshed_vol[30,:,:,:], dim='z')

interactive(children=(IntSlider(value=92, description='slice', max=184), Output()), _dom_classes=('widget-inte…

## Skull Stripping T1

In [74]:
t1_brain_vol = create_normalized_volume(skull_strip_anat(t1_image, mni_template, mni_mask).numpy())
# lil redundant as it's already normalized, but we'll fix that when we clean this up

Skull Stripping Anatomy Volume
Registering template to frame
Creating brain mask
Dilating brain mask
Masking brain
Done


In [75]:
explore_3D_vol(t1_brain_vol[:,:,:], dim='x', cmap='gray')

interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

# Method of Subtraction

Don't forget the rest file is here:
/Users/joachimpfefferkorn/repos/neurovolume/media/sub-01/func/sub-01_task-rest_bold.nii.gz


**you need to start using it**


**Should these functions, or any future "more correct" control subtraction functions use absolute**

In [58]:
def subtract_previous_frame(bold_vol: np.ndarray):
    result = np.empty_like(bold_vol)
    for frame in range(bold_vol.shape[3]):
        if frame == range(og_bold_vol.shape[3])[0]:
            result[:,:,:,frame] = np.zeros_like(bold_vol[:,:,:,frame])
        else:
            result[:,:,:,frame] = bold_vol[:,:,:,frame] - bold_vol[:,:,:,frame-1]
    return np.abs(result)


In [59]:
def subtract_control_frame(bold_vol: np.ndarray, control_frame_idx: int):
    result = np.empty_like(bold_vol)
    for frame in range(bold_vol.shape[3]):
        result[:,:,:,frame] = bold_vol[:,:,:,frame] - bold_vol[:,:,:,control_frame_idx]
    return np.abs(result)

In [66]:
print(bold_vol.shape[0])

64


In [69]:
def subtract_average_from_frame(bold_vol: np.ndarray, alignment_frame = False):
    """
    An alignment frame is the average bold frame tacked on the last frame
    You can use this for manually aligning to anatomy scans
    """
    average = np.mean(bold_vol, axis=3)
    if alignment_frame:
        seq_len = bold_vol.shape[3]+1
    else:
        #result = np.empty_like(bold_vol)
        seq_len = bold_vol.shape[3]

    result = np.empty([bold_vol.shape[0], bold_vol.shape[1], bold_vol.shape[2], seq_len])

    for frame in range(bold_vol.shape[3]):
        result[:,:,:,frame] = bold_vol[:,:,:,frame] - average
    
    if alignment_frame:
        result[:,:,:,seq_len-1] = average

    return np.abs(result)

In [70]:
diffed_bold = subtract_average_from_frame(bold_threshed_vol, alignment_frame=True)

In [71]:
explore_3D_vol(diffed_bold[30,:,:,:], dim='z', cmap="gray")

interactive(children=(IntSlider(value=92, description='slice', max=185), Output()), _dom_classes=('widget-inte…

# Exports

In [19]:
with open(f'{output_folder}/mask.npy', 'wb') as f:
    np.save(f, np.array(mask_vol))

In [78]:
with open(f'{output_folder}/anat.npy', 'wb') as f:
    np.save(f, np.array(og_t1_vol))

In [76]:
with open(f'{output_folder}/bold.npy', 'wb') as f:
    np.save(f, np.array(diffed_bold))

In [77]:
with open(f'{output_folder}/anat_brain.npy', 'wb') as f:
    np.save(f, np.array(t1_brain_vol))